## **Predicting Body Mass Index**

Aim:    PANDAS, NUMPY, sqlalchemy,pyodbc, PANDAS_PROFILING, SEABORN, MATPLOTLIB, SKLEARN, XGBOOST, KAGGLE

ensure font is the same on dashboard and presentation 
make dashboard legends square??? 


TAKE GENDER OUT, ITS NOT DOING ANYTHING WITHIN THE MODEL! 

In [2]:
# Importing project dependencies 

# General 
import pandas as pd 
import numpy as np

# Data warehousing 
from sqlalchemy import create_engine
import pyodbc
import urllib

# Exploratory Data Analysis & Visualisation 
%matplotlib inline
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt

# Feature Engineering and Modelling 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier

# Model imporvement 
from sklearn.metrics import accuracy_score,  classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn import metrics
import math

# Model export for deployment 
import pickle

#### **----------------------------------  1. Data collection  -----------------------------------**

In [3]:
# Importing data through API 
!kaggle datasets download -d yasserh/bmidataset -p ..\Data --unzip 


  0%|          | 0.00/2.19k [00:00<?, ?B/s]
100%|██████████| 2.19k/2.19k [00:00<00:00, 2.24MB/s]


In [4]:
# Reading data into dataframe 
data = pd.read_csv('../Data/bmi.csv')

#### **----------------------------------  2. Data pre-processing  -----------------------------------**

In [5]:
# Viewing first 5 rows of data
data.head()

,Gender,Height,Weight,Index
0,Male,174,96,4
1,Male,189,87,2
2,Female,185,110,4
3,Female,195,104,3
4,Male,149,61,3


Formula: weight (kg) / [height (m)]2

In [6]:
%%capture 
# Supressing cell run output 

"""
BMI	Condition 
# Below 18.5   == Underweight        ----  INDEX == 0
# 18.5-24.9    == Normal weight      ----  INDEX == 1
# 25-29.9	   == Overweight         ----  INDEX == 2
# 30-39.9      == Obese              ----  INDEX == 3 
# 40 and more  == Severely Obese     ----  INDEX == 4
"""

In [7]:
# Correcting BMI Indexes (Divided by 100 as height came orignally in cm)
data['newIndex'] = data['Weight'] / (data['Height']/100)**2

In [8]:
# Viewing first 5 rows of data
data.head()

,Gender,Height,Weight,Index,newIndex
0,Male,174,96,4,31.708284
1,Male,189,87,2,24.355421
2,Female,185,110,4,32.140248
3,Female,195,104,3,27.350427
4,Male,149,61,3,27.476240


In [9]:
# BMI conditions to correct the data index to UK(NHS) classifications
def set_bmi(row):
    if row["newIndex"] < 18.5:
        return 0
    elif row["newIndex"] >= 18.5 and  row["newIndex"] <= 24.999 :
        return 1
    elif row["newIndex"] >= 25 and  row["newIndex"] <= 29.999 :
        return 2
    elif row["newIndex"] >= 30 and  row["newIndex"] <= 39.999 :
        return 3
    else:
        return 4

# Using assign to add new column BMI based on the function conditions
data = data.assign(BMI=data.apply(set_bmi, axis=1))

pd.options.mode.chained_assignment = None  # Supressing SettingWithCopyWarning message

# Creating 'bmiclass' field 
data['bmiclass'] = ''

# Ouputing labelled column based on bmi of individual
data.bmiclass[data['BMI'] == 0] = 'Underweight'
data.bmiclass[data['BMI'] == 1] = 'Normal weight'
data.bmiclass[data['BMI'] == 2] = 'Overweight'
data.bmiclass[data['BMI'] == 3] = 'Obese'
data.bmiclass[data['BMI'] == 4] = 'Severely obese'

# Saving cleaned data as csv for visualisation
data.to_csv('../Data/bmi_clean.csv', index=False)

# Dropping unnecessary columns
data.drop(['Index','newIndex'], axis=1, inplace=True)

In [10]:
# Viewing first 5 rows of data
data.head()

,Gender,Height,Weight,BMI,bmiclass
0,Male,174,96,3,Obese
1,Male,189,87,1,Normal weight
2,Female,185,110,3,Obese
3,Female,195,104,2,Overweight
4,Male,149,61,2,Overweight


In [11]:
# Viewing the data types of the columns
data.dtypes

Gender      object
Height       int64
Weight       int64
BMI          int64
bmiclass    object
dtype: object

In [12]:
# Viewing dataset shape
data.shape

(500, 5)

In [13]:
# 1st check for null values and datatype check 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Gender    500 non-null    object
 1   Height    500 non-null    int64 
 2   Weight    500 non-null    int64 
 3   BMI       500 non-null    int64 
 4   bmiclass  500 non-null    object
dtypes: int64(3), object(2)
memory usage: 19.7+ KB


In [14]:
# 2nd check for null values
data.isnull().sum()

Gender      0
Height      0
Weight      0
BMI         0
bmiclass    0
dtype: int64

In [15]:
# Unique count for each variable
data.nunique()

Gender        2
Height       60
Weight      110
BMI           5
bmiclass      5
dtype: int64

In [16]:
# Viewing summary of statistics for numeric columns
data.describe()

,Height,Weight,BMI
count,500.000000,500.000000,500.000000
mean,169.944000,106.000000,2.806000
std,16.375261,32.382607,1.280486
min,140.000000,50.000000,0.000000
25%,156.000000,80.000000,2.000000
50%,170.500000,106.000000,3.000000
75%,184.000000,136.000000,4.000000
max,199.000000,160.000000,4.000000


In [17]:
# (SQL standard) Formatting column headers by removing potential capital letters and spaces in column headers 
data.columns = data.columns.str.lower()
data.columns = data.columns.str.replace(' ','_')

#### **----------------------------------  3. Data Warehousing  -----------------------------------**

In [18]:
# Saving cleaned data as csv
data.to_csv('../Data/bmi_cleanML.csv', index=False)

In [19]:
# def ETL_SQLServer(Server,db,tablename):
#   """
#     :param Server: Destination Server
#     :param db: Destination Database name MUST ALREADY EXIST
#     :param tablename: Project ID and Dataset name
#     """

#   # SQL table header format
#   tablename = tablename.lower()
#   tablename = tablename.replace(' ','_')

#   quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=%s;DATABASE=%s"%(Server,db))
#   engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

#   data.to_sql('{tb}'.format(tb=tablename), schema='dbo', con = engine, index=False, if_exists='replace')

#   return("ETL successful, {num} rows loaded into table: {tb}.".format(num=len(data.iloc[:,0]), tb=tablename))

In [20]:
# # Function to warehouse dataframe
# ETL_SQLServer("[SERVER NAME]","Projects","P11 bmi")

#### **----------------------------------  4. Exploratory data analysis  -----------------------------------**

In [21]:
# Checking distribution of categorical fields 
print(data.gender.value_counts())
print(data.bmi.value_counts())

Female    255
Male      245
Name: gender, dtype: int64
4    203
3    130
2     68
1     65
0     34
Name: bmi, dtype: int64


In [22]:
# Viewing summary of statistics for numeric columns
data.describe()

,height,weight,bmi
count,500.000000,500.000000,500.000000
mean,169.944000,106.000000,2.806000
std,16.375261,32.382607,1.280486
min,140.000000,50.000000,0.000000
25%,156.000000,80.000000,2.000000
50%,170.500000,106.000000,3.000000
75%,184.000000,136.000000,4.000000
max,199.000000,160.000000,4.000000


In [23]:
# Displaying the distribution of data based on a five number summary (“minimum”, first quartile (Q1), median, third quartile (Q3), and “maximum”). 
chart = data.boxplot(column = ['height', 'weight'])
chart.set_xticklabels(chart.get_xticklabels(), rotation = 0)
plt.title('The Factor of Body Mass Index', fontsize = 15, color = 'maroon', pad = 12)
plt.xlabel('Factors')
plt.ylabel('Count')
plt.show()

C:\Users\matti\AppData\Local\Temp/ipykernel_10928/495980808.py:7: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [24]:
%%capture 
# Supressing cell run output 

"""
# Viewing data distribution of 'height' column
sns.boxplot(x='height', data=data, color='yellow')

# Removing the spines from the right and upper portion of the plot by default.
sns.despine()
"""

In [25]:
%%capture 
# Supressing cell run output 

"""
# Viewing data distribution of 'weight' column
sns.boxplot(x='weight', data=data, color='green')

# Removing the spines from the right and upper portion of the plot by default.
sns.despine()
"""

In [26]:
# PairPlot to show relationships in a dataset. *(To view list of palettes enter random value, all palettes will be shown in error message)
plot = sns.pairplot(data, vars=['height', 'weight'], 
             hue="bmi", palette='RdYlGn', diag_kind = 'kde',
             plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},
             height = 4)

# Setting plot title 
plt.suptitle('Pair Plot of Height and Weight Correlation to BMI', 
             size = 20, y=1.04);

# Removing the spines from the right and upper portion of the plot by default.
sns.despine()

In [27]:
pd.options.mode.chained_assignment = None  # Supressing SettingWithCopyWarning message

# Creating 'bmiclass' field 
data['bmiclass'] = ''

# Ouputing labelled column based on bmi of individual
data.bmiclass[data['bmi'] == 0] = 'Underweight'
data.bmiclass[data['bmi'] == 1] = 'Normal weight'
data.bmiclass[data['bmi'] == 2] = 'Overweight'
data.bmiclass[data['bmi'] == 3] = 'Obese'
data.bmiclass[data['bmi'] == 4] = 'Severely obese'

In [28]:
# Saving cleaned data as csv
data.to_csv('../Data/bmi_clean.csv', index=False)

In [29]:
# Calculations for bar chart 
Underweight = data[data['bmiclass'] == 'Underweight']['bmi'].count() / data.shape[0] * 100
Normalweight = data[data['bmiclass'] == 'Normal weight']['bmi'].count() / data.shape[0] * 100
Overweight = data[data['bmiclass'] == 'Overweight']['bmi'].count() / data.shape[0] * 100
Obese = data[data['bmiclass'] == 'Obese']['bmi'].count() / data.shape[0] * 100
Severelyobese = data[data['bmiclass'] == 'Severely obese']['bmi'].count() / data.shape[0] * 100


In [30]:
# Showing distribtuon of target values 
fig, ax = plt.subplots()
sns.countplot(x='bmi', palette="Set3", data=data)
bmiclasses = ['Underweight','Healthy','Overweight', 'Obese','Severely obese']
plt.xticks([0, 1, 2, 3, 4], bmiclasses)
plt.xlabel('BMI', size=15, labelpad=12, color='grey')
plt.ylabel('Count', size=15, labelpad=12, color='grey')
plt.title("Distribution of BMI in dataset", size=15, pad=20)
plt.ylim(0, 250)
plt.text(-0.2, 10, f"{round(Underweight, 2)}%", fontsize=12)
plt.text(0.75, 10, f"{round(Normalweight, 2)}%", fontsize=12)
plt.text(1.75, 10, f"{round(Overweight, 2)}%", fontsize=12)
plt.text(2.75, 10, f"{round(Obese, 2)}%", fontsize=12)
plt.text(3.75, 10, f"{round(Severelyobese, 2)}%", fontsize=12)
sns.despine()
plt.show()

C:\Users\matti\AppData\Local\Temp/ipykernel_10928/2039109783.py:16: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [31]:
# Calculations for bar chart 
Male = data[data['gender'] == 'Male']['bmi'].count() / data.shape[0] * 100
Female = data[data['gender'] == 'Female']['bmi'].count() / data.shape[0] * 100

In [32]:
# Showing distribtuon of target values 
fig, ax = plt.subplots()
sns.countplot(x='gender',hue="bmi", palette="Set3", data=data)
gender = ['Male','Female']
plt.xticks([0, 1], gender)
plt.xlabel('BMI', size=15, labelpad=12, color='grey')
plt.ylabel('Count', size=15, labelpad=12, color='grey')
plt.title("Distribution of BMI by gender", size=15, pad=20)
plt.ylim(0, 250)
plt.text(-0.2, 150, f"{round(Male, 2)}%", fontsize=12)
plt.text(0.75, 150, f"{round(Female, 2)}%", fontsize=12)
sns.despine()
plt.show()

C:\Users\matti\AppData\Local\Temp/ipykernel_10928/351321948.py:13: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [33]:
#correlation of each variable
data.corr()

,height,weight,bmi
height,1.000000,0.000446,-0.416359
weight,0.000446,1.000000,0.812227
bmi,-0.416359,0.812227,1.000000


In [34]:
# Viewing correlations of each features in dataset 
corrmat = data.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(8,8))

# Plotting heat map
plot = sns.heatmap(data[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [35]:
# PANDAS PROFILING

# profile report of each variable
# pandas_profiling.ProfileReport(data)

# We only use the first 100 data points
# prof = pandas_profiling.ProfileReport(data.sample(n=100)) 
# prof.to_file(output_file='output.html')

#### **----------------------------------  5. Feature Engineering  -----------------------------------**

In [36]:
# Dropping fields that wont benefit the model
data.drop(['bmiclass'], axis=1,inplace=True)

In [37]:
# Viewing first 5 rows of data 
data.head()

,gender,height,weight,bmi
0,Male,174,96,3
1,Male,189,87,1
2,Female,185,110,3
3,Female,195,104,2
4,Male,149,61,2


In [38]:
# One hot encoding for encoding categorical variables
final_data = pd.get_dummies(data, drop_first=True)

In [39]:
# Viewing final dataset for model 
final_data

,height,weight,bmi,gender_Male
0,174,96,3,1
1,189,87,1,1
2,185,110,3,0
3,195,104,2,0
4,149,61,2,1
...,...,...,...,...
495,150,153,4,0
496,184,121,3,0
497,141,136,4,0
498,150,95,4,1


In [40]:
# Splitting dependent and independent features
X = final_data.drop('bmi', axis = 1)
y = final_data['bmi']

In [41]:
# Viewing first 5 rows of X
X.head()

,height,weight,gender_Male
0,174,96,1
1,189,87,1
2,185,110,0
3,195,104,0
4,149,61,1


In [42]:
# Using train test split to split train and test data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 23)

# Viewing shape of train / test data
print(X_train.shape)
print(X_test.shape)

(400, 3)
(100, 3)


In [43]:
# Scaling 
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

#### **----------------------------------  6. Modelling  -----------------------------------**

In [44]:
# Preparing algorithms in a list
random_state = 23
classifiers = [
    KNeighborsClassifier(n_neighbors = 5),
    LinearSVC(),
    DecisionTreeClassifier(criterion = 'gini', max_depth = 3),
    RandomForestClassifier(max_features = 'sqrt', n_jobs = 1, verbose = 1),
    XGBClassifier(use_label_encoder=False),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [45]:
pd.options.mode.chained_assignment = None  # Supressing SettingWithCopyWarning

# Creating a df to show models and their accuracy 
log_cols = ["Classifier", "Accuracy"]
log = pd.DataFrame(columns = log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    print("=" * 30)
    print(name)
    print('****  Results  ****')
    
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    print("\n")
    
    # Creating new df and appending results onto initial df 
    log_entry = pd.DataFrame([[name, acc * 100]], columns = log_cols)
    log = log.append(log_entry)
    
print("=" * 30)

KNeighborsClassifier
****  Results  ****
Accuracy: 89.0000%


LinearSVC
****  Results  ****
Accuracy: 66.0000%


DecisionTreeClassifier
****  Results  ****
Accuracy: 69.0000%


RandomForestClassifier
****  Results  ****
Accuracy: 89.0000%




[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


[23:34:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier
****  Results  ****
Accuracy: 87.0000%


AdaBoostClassifier
****  Results  ****
Accuracy: 41.0000%


GaussianNB
****  Results  ****
Accuracy: 68.0000%


QuadraticDiscriminantAnalysis
****  Results  ****
Accuracy: 95.0000%




#### **----------------------------------  7. Cross Validation -----------------------------------**

In [46]:
# Visualising accuracy models
plt.figure(figsize = (10,6))
sns.set_color_codes("muted")
sns.barplot(x = 'Accuracy', y = 'Classifier', data = log, palette="rainbow")
plt.xlabel('Accuracy %')
plt.title('Accuracy Score of Classification Model')
plt.show()

C:\Users\matti\AppData\Local\Temp/ipykernel_10928/3522384902.py:7: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [47]:
# Cross validate model with Kfold stratified cross validation
kfold = StratifiedKFold(n_splits=5)

# Modeling step Test differents algorithms 
cv_results = []
for classifier in classifiers :
    cv_results.append(cross_val_score(classifier, X_train, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=4))

cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["KNN","LinearSVC","DecisionTree","Random Forrest","XGBClassifier","AdaBoost","GaussianNB","QuadraticDA"]})

g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")

C:\Users\matti\anaconda3\envs\P11\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


#### **----------------------------------  8. Evaluation -----------------------------------**

We will determine the parameters and values that we want to search in each of our models. Then, we will perform the GridSearchCV and set the best parameters obtained in our model.

In [48]:
def print_best_model(model):
    """
    Prints the best parameteres, score and estimator for the model given.
    """
    print(f"The best parameters are: {model.best_params_}")
    print(f"The best model score is: {model.best_score_}")    
    print(f"The best estimator is: {model.best_estimator_}")

**QuadraticDiscriminantAnalysis**

In [49]:
# Defining parameters for GridSearchCV
params = [{'reg_param': [0.1, 0.2,0.23,0.24,0.25,0.254,0.255,0.256,0.26,0.27,0.28,0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1.1,1.2]}]

# Initialising algorithm 
qda = QuadraticDiscriminantAnalysis()

# Outputting best param with GridDearch
qda_best = GridSearchCV(qda, params,cv=4)

# Fitting best paramters to X and y
qda_best.fit(X,y)

C:\Users\matti\anaconda3\envs\P11\lib\site-packages\sklearn\discriminant_analysis.py:903: RuntimeWarning: invalid value encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\matti\anaconda3\envs\P11\lib\site-packages\sklearn\discriminant_analysis.py:906: RuntimeWarning: invalid value encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\matti\anaconda3\envs\P11\lib\site-packages\sklearn\discriminant_analysis.py:903: RuntimeWarning: invalid value encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\matti\anaconda3\envs\P11\lib\site-packages\sklearn\discriminant_analysis.py:906: RuntimeWarning: invalid value encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\matti\anaconda3\envs\P11\lib\site-packages\sklearn\discriminant_analysis.py:903: RuntimeWarning: invalid value encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\matti\anaconda3\envs\P11\lib\site-packages\sklearn\disc

GridSearchCV(cv=4, estimator=QuadraticDiscriminantAnalysis(),
             param_grid=[{'reg_param': [0.1, 0.2, 0.23, 0.24, 0.25, 0.254,
                                        0.255, 0.256, 0.26, 0.27, 0.28, 0.3,
                                        0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.1,
                                        1.2]}])

In [50]:
# Calling function to output best parameters 
print_best_model(qda_best)

The best parameters are: {'reg_param': 0.25}
The best model score is: 0.964
The best estimator is: QuadraticDiscriminantAnalysis(reg_param=0.25)


In [51]:
# Fitting best model
model = QuadraticDiscriminantAnalysis(reg_param=0.25)
model.fit(X_train, y_train)

QuadraticDiscriminantAnalysis(reg_param=0.25)

In [52]:
# Predicting using X_train 
qda_predict_train = model.predict(X_train)

In [53]:
# Checking the accuracy score 
accuracy_score(y_train, qda_predict_train)

0.8325

In [54]:
# Viewing classification report
print('Classification Report of Quadratic Discriminant Analysis: \n')
print(classification_report(y_train, qda_predict_train))

Classification Report of Quadratic Discriminant Analysis: 

              precision    recall  f1-score   support

           0       0.78      0.85      0.82        34
           1       0.73      0.61      0.67        44
           2       0.69      0.42      0.52        52
           3       0.74      0.98      0.85       109
           4       0.99      0.92      0.95       161

    accuracy                           0.83       400
   macro avg       0.79      0.76      0.76       400
weighted avg       0.84      0.83      0.82       400



In [55]:
# Predicting using X_test
qda_predict_test = model.predict(X_test)

In [56]:
# Checking the accuracy score 
accuracy_score(y_test, qda_predict_test)

0.72

In [57]:
# Viewing classification report
print('Classification Report of Quadratic Discriminant Analysis: \n')
print(classification_report(y_test, qda_predict_test))


Classification Report of Quadratic Discriminant Analysis: 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.92      0.52      0.67        21
           2       0.50      0.31      0.38        16
           3       0.55      1.00      0.71        21
           4       1.00      0.83      0.91        42

    accuracy                           0.72       100
   macro avg       0.59      0.53      0.53       100
weighted avg       0.81      0.72      0.73       100



C:\Users\matti\anaconda3\envs\P11\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\matti\anaconda3\envs\P11\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\matti\anaconda3\envs\P11\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
# Plotting confusion matrix
matrix = pd.DataFrame((metrics.confusion_matrix(y_test, qda_predict_test)), 
                      (0, 1, 2, 3, 4), 
                      (0, 1, 2, 3, 4))
print(matrix)

#visualize confusion matrix
plt.figure(figsize = (8,5))
heatmap = sns.heatmap(matrix, annot = True, annot_kws = {'size': 14}, fmt = 'd', cmap = 'YlGnBu')
heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation = 0, ha = 'right', fontsize = 14)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation = 0, ha = 'right', fontsize = 14)

plt.title('Confusion Matrix\n(Quadratic Discriminant Analysis)', fontsize = 18, color = 'darkblue')
plt.ylabel('True label', fontsize = 14)
plt.xlabel('Predicted label', fontsize = 14)
plt.show()

   0   1  2   3   4
0  0   0  0   0   0
1  5  11  5   0   0
2  0   1  5  10   0
3  0   0  0  21   0
4  0   0  0   7  35


C:\Users\matti\AppData\Local\Temp/ipykernel_10928/3428842729.py:16: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [59]:
# Error checking of MSE & RMSE 
mse = metrics.mean_squared_error(y_test,qda_predict_test)
print('Mean Squared Error : '+ str(mse))
rmse = math.sqrt(metrics.mean_squared_error(y_test, qda_predict_test))
print('Root Mean Squared Error : '+ str(rmse))

Mean Squared Error : 0.28
Root Mean Squared Error : 0.5291502622129182


In [60]:
# Showing difference between y-test value and results from X_test
sns.distplot(y_test-qda_predict_test)

C:\Users\matti\anaconda3\envs\P11\lib\site-packages\seaborn\distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<AxesSubplot:title={'center':'Confusion Matrix\n(Quadratic Discriminant Analysis)'}, xlabel='bmi', ylabel='True label'>

In [61]:
# Plot is linear so prediction is fairly accurate 
plt.scatter(y_test, qda_predict_test)
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title(" Actual Prices vs Predicted Prices")
plt.show()

C:\Users\matti\AppData\Local\Temp/ipykernel_10928/3975312387.py:6: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [62]:
# Fitting the model 
model.fit(X, y)

QuadraticDiscriminantAnalysis(reg_param=0.25)

#### **----------------------------------  9. Exporting Model -----------------------------------**

In [69]:
# dump information to that file
pickle.dump(model, open('.././_analysis.pkl', 'wb'))
../../

In [71]:
# update absolute and relative paths file
# remove gender from model 